In [1281]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

from pysistence import make_list #pip install pysistence

In [1282]:
def normalizacion(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [1283]:
class Car(ap.Agent):
    """ An agent with a position and velocity in a continuous space,
    who follows Craig Reynolds three rules of flocking behavior;
    plus a fourth rule to avoid the edges of the simulation space. """

    def setup(self):

        self.velocity =  [1.0, 0.000] 
        self.acceleration = [0 , 0]          
        self.close_car = False
        self.random = self.model.random
        self.group = self.random.choice(range(100))
        print(self.group)
        if self.group < self.p.problemas:
            self.error = ((self.random.choice(range(self.p.intensidad_problemas))-self.p.intensidad_problemas/2)/self.p.intensidad_problemas)
        else:
            self.error = 0
        print(self.error)    

    def setup_pos(self, space):

        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def update_velocity(self):

        pos = self.pos
        ndim = self.p.ndim

        nbs = self.neighbors(self, distance=self.p.outer_radius)
        nbs_len = 0
        nbs_pos_array = np.array(nbs.pos)
        v1 = np.zeros(ndim)
        for nbs_check in nbs_pos_array:
            if (nbs_check[0] -  pos[0]  > -1 and nbs_check[0] -  pos[0]  < self.p.outer_radius ) and (nbs_check[1] -  pos[1]  > -1 and nbs_check[1] -  pos[1]  < 1 ) :
                nbs_len += 1
        nbs_vec_array = np.array(nbs.velocity)
        if nbs_len > 0:
            if self.velocity[0] > 0.0:
                self.velocity =   [0, 0]   
            else:
                v1 = np.zeros(ndim)
        else:
            if self.velocity[0] < 1.0:
                v1 = np.array([1, 0.])
            else:
                v1 = np.zeros(ndim)

        
    
        self.velocity += v1
        self.velocity = normalizacion(self.velocity)+normalizacion(self.velocity)*self.error
        if pos[0] > 99:
            self.velocity = [1,0]

    def update_position(self):

        self.space.move_by(self, self.velocity)


    def remove_agent(self):
        """ Removes agents from the space. """
        
        #if self.pos[0] > 90:
    
        

In [1284]:
[1.0, 0.000][1]

0.0

In [1285]:
class CarIncor(ap.Agent):
    """ An agent with a position and velocity in a continuous space,
    who follows Craig Reynolds three rules of flocking behavior;
    plus a fourth rule to avoid the edges of the simulation space. """

    def setup(self):

        self.velocity =  [1.0, 0.1] 
        self.acceleration = [0 , 0]          
        self.car_incor = False
        self.random = self.model.random
        self.group = self.random.choice(range(100))
        print(self.group)
        if self.group < self.p.problemas:
            self.error = ((self.random.choice(range(self.p.intensidad_problemas))-self.p.intensidad_problemas/2)/self.p.intensidad_problemas)
        else:
            self.error = 0
        print(self.error)    

    def setup_pos(self, space):

        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def update_velocity(self):

        pos = self.pos
        ndim = self.p.ndim

        

        nbs = self.neighbors(self, distance=self.p.outer_radius)
        nbs_len = 0
        nbs_pos_array = np.array(nbs.pos)
        for nbs_check in nbs_pos_array:
            if pos[1] >= 5.0:
                if (nbs_check[0] -  pos[0]  > 0 and nbs_check[0] -  pos[0]  < self.p.outer_radius ) and (nbs_check[1] -  pos[1]  > -1 and nbs_check[1] -  pos[1]  < 1 ) :
                    nbs_len += 1
            else:
                if (nbs_check[0] -  pos[0]  > 1 and nbs_check[0] -  pos[0]  < self.p.inner_radius*0.5 ) and (nbs_check[1] -  pos[1]  > -1 and nbs_check[1] -  pos[1]  < 1 ) :
                    nbs_len += 1

        #nbs_vec_array = np.array(nbs.velocity)
        v1 = np.zeros(ndim)
        if pos[1] >= 5.0:
            if not self.car_incor:
                self.velocity = [1, 0.] # Cambiar 10 por velocidad a la cual se tiene que disminuir
                self.car_incor = True
            if nbs_len > 0:
                if self.velocity[0] > 0.0:
                    self.velocity =  [0, 0]   
                else:
                    v1 = np.zeros(ndim)
            else:
                if self.velocity[0] < 1.0:
                    v1 = np.array([1.0, 0.])
                else:
                    v1 = np.zeros(ndim)
        else:
            if nbs_len > 0:
                if self.velocity[0] > 0.0:
                    self.velocity =   [0, 0]   
            else:
                if self.velocity[0] < 10.0:
                    self.velocity =   [1.0, 0.1]  


        self.velocity += v1
        self.velocity = normalizacion(self.velocity)+normalizacion(self.velocity)*self.error

        if pos[0] > 99:
            self.velocity = [1,0]
        
    def update_position(self):

        self.space.move_by(self, self.velocity)
        if self.pos[1]>=5.0:
            self.pos[1]=5.0
            #del self.space.positions[self]
        

In [1286]:
class MergeCar(ap.Agent):
    pass

In [1287]:
def parse_positions(positions: list) -> list[tuple]:
    """Return paramater positions as a list of tuples so we can use add_agents method."""
    pass    


def initialize_starting_positions(car_amount) -> list[tuple]:
    """Return a list of tuples as starting position for cars."""
    pass

In [1288]:
class TrafficFlowModel(ap.Model):
    

    def setup(self):
        """ Initializes the agents and network of the model. """


        if self.p.cars_pos:
            # Initialize model with existing car positions
            self.space = ap.Space(self, shape=[self.p.sizeX+self.p.outer_radius+5,self.p.sizeY])
            self.agents = ap.AgentDList(self, self.p.population, Car)
            self.agentsIncor = ap.AgentDList(self, self.p.populationIncor, CarIncor)
            self.space.add_agents(self.agents,self.p.cars_pos, random=False)
            self.space.add_agents(self.agentsIncor,self.p.cars_posIncor, random=False)
            self.agents.setup_pos(self.space)
            self.agentsIncor.setup_pos(self.space)
            self.carrosNew = False
            self.carrosNewPop = 0
            self.carrosNewList = []

        else:
            # Stop simulation and return error
            pass

    def step(self):

        # Update velocity and position for cars in every step

        self.agents.update_velocity()  # Adjust direction
        self.agentsIncor.update_velocity() 
        self.agents.update_position()  # Move into new direction
        self.agentsIncor.update_position() 
        if self.carrosNew:
            for agentNew in self.carrosNewList:
                agentNew.update_velocity() 
                agentNew.update_position() 
        
        #self.remove_agent(self.agentsIncor)
        print(f"Iteracion t={self.t}")
        if self.t % 5 == 0 and self.t != 0:
            generacion = self.random.choice(range(100))
            if generacion < self.p.densidad:
                carrosEnInicio = 0
                for agent in self.agents:
                    if (agent.pos[0]  < self.p.outer_radius+1 ) :
                        carrosEnInicio += 1
                if self.carrosNew:
                    for carros in self.carrosNewList:
                        for agent in carros:
                            if (agent.pos[0]  < self.p.outer_radius+1 ) :
                                carrosEnInicio += 1
                if carrosEnInicio == 0 and self.carrosNew:
                    self.carrosNewList.append(ap.AgentDList(self, 1, Car))
                    self.space.add_agents(self.carrosNewList[len(self.carrosNewList)-1],[np.array([1, 5.])], random=False)
                    self.carrosNewList[len(self.carrosNewList)-1].setup_pos(self.space)
                elif carrosEnInicio == 0:
                    self.carrosNewList.append(ap.AgentDList(self, 1, Car))
                    self.space.add_agents(self.carrosNewList[0],[np.array([1, 5.])], random=False)
                    self.carrosNewList[0].setup_pos(self.space)
                    self.carrosNew = True

        
        if self.t % 5 == 0 and self.t != 0:
            generacion = self.random.choice(range(100))
            if generacion < self.p.densidad_incor:
                carrosEnInicio = 0
                for agent in self.agents:
                    if (agent.pos[0]  < self.p.outer_radius+21 and agent.pos[0]  > 18 ) and (agent.pos[1]  < 4 and agent.pos[1]  > 0 ) :
                        carrosEnInicio += 1
                if self.carrosNew:
                    for carros in self.carrosNewList:
                        for agent in carros:
                            if (agent.pos[0]  < self.p.outer_radius+21 and agent.pos[0]  > 18 ) and (agent.pos[1]  < 4 and agent.pos[1]  > 0 ) :
                                carrosEnInicio += 1
                if carrosEnInicio == 0 and self.carrosNew:
                    self.carrosNewList.append(ap.AgentDList(self, 1, CarIncor))
                    self.space.add_agents(self.carrosNewList[len(self.carrosNewList)-1],[np.array([20, 2.])], random=False)
                    self.carrosNewList[len(self.carrosNewList)-1].setup_pos(self.space)
                elif carrosEnInicio == 0:
                    self.carrosNewList.append(ap.AgentDList(self, 1, CarIncor))
                    self.space.add_agents(self.carrosNewList[0],[np.array([20, 2.])], random=False)
                    self.carrosNewList[0].setup_pos(self.space)
                    self.carrosNew = True
                    self.model.random

    


        # Check if any of the cars have a car nearby. If so, update agent.close_car to True

        # Update VSL value depending on density of cars in one specific area in grid.

    

    def calculate_VSL(self):
        # Calculate VSL inside a specific range
        pass

In [1289]:
def animation_plot_single(m, ax):
    ndim = m.p.ndim
    ax.set_title(f"Filas de vehiculos en movimiento {ndim}D t={m.t}")
    pos = m.space.positions.values()
    pos = np.array(list(pos)).T  # Transform
    ax.scatter(*pos, s=1, c='black')
    ax.set_xlim(0, m.p.sizeX)
    ax.set_ylim(0, m.p.sizeY)
    if ndim == 3:
        ax.set_zlim(0, m.p.size)
    ax.set_axis_off()

def animation_plot(m, p):
    projection = '3d' if p['ndim'] == 3 else None
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_subplot(111, projection=projection)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20))

In [1290]:
parameters2D = {
    'sizeX': 100,
    'sizeY': 10,
    'seed': 123,
    'steps': 150,
    'ndim': 2,
    'population': 5,
    'populationIncor': 2,
    'inner_radius': 2,
    'outer_radius': 4,
    'problemas':5,
    'intensidad_problemas':20,
    'densidad':50,
    'densidad_incor':30,
    'cars_pos':[np.array([0, 5.]),np.array([6, 5.]),np.array([12, 5.]),np.array([18, 5.]),np.array([24, 5.])],
    'cars_posIncor':[np.array([20, 2.]),np.array([30, 3.])]
}

In [1291]:
animation_plot(TrafficFlowModel, parameters2D)

52
0
34
0
13
0
4
0.1
68
0
71
0
42
0
Iteracion t=1
Iteracion t=2
Iteracion t=3
Iteracion t=4
Iteracion t=5
6
0
17
0
Iteracion t=6
Iteracion t=7
Iteracion t=8
Iteracion t=9
Iteracion t=10
71
0
Iteracion t=11
Iteracion t=12
Iteracion t=13
Iteracion t=14
Iteracion t=15
Iteracion t=16
Iteracion t=17
Iteracion t=18
Iteracion t=19
Iteracion t=20
0
0.15
Iteracion t=21
Iteracion t=22
Iteracion t=23
Iteracion t=24
Iteracion t=25
76
0
Iteracion t=26
Iteracion t=27
Iteracion t=28
Iteracion t=29
Iteracion t=30
0
0.0
Iteracion t=31
Iteracion t=32
Iteracion t=33
Iteracion t=34
Iteracion t=35
5
0
Iteracion t=36
Iteracion t=37
Iteracion t=38
Iteracion t=39
Iteracion t=40
85
0
Iteracion t=41
Iteracion t=42
Iteracion t=43
Iteracion t=44
Iteracion t=45
2
-0.05
Iteracion t=46
Iteracion t=47
Iteracion t=48
Iteracion t=49
Iteracion t=50
Iteracion t=51
Iteracion t=52
Iteracion t=53
Iteracion t=54
Iteracion t=55
60
0
98
0
Iteracion t=56
Iteracion t=57
Iteracion t=58
Iteracion t=59
Iteracion t=60
43
0
Iteracion